In [3]:
import os
import pandas as pd
import numpy as np
import requests
import io
import json
from datetime import datetime

## This Cell pulls reservoir level data from the Sea Level center Reservoir Database
Home page link
- https://uhslc.soest.hawaii.edu/

Data Dashboard 
- https://uhslc.soest.hawaii.edu/reservoir/

In [ ]:
# Set up the loacation of where to put the data
today = datetime.today().strftime('%Y-%m-%d')            # new data for each date so record the date
os.makedirs(os.path.dirname(json_file), exist_ok=True)   # create a new folder for todays date 
json_file = os.path.join("..", "Data/GIS/Rez_locations", "stations.geojson-{}.txt".format(today))
